### Imports

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchinfo import summary as Model_Summary
import torch.optim as optim

PyTorch Version:  2.0.0+cu117
Torchvision Version:  0.15.1+cu117


### Inception Model

Convolutional block: 1 convolutional layer + batch normalisation, with ReLU activation

- include kernel size and stride here?
- is environment.yaml file .yaml or .yml?

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_chanels, kernel_size, stride, **kwargs):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_chanels, kernel_size, stride, **kwargs)
        self.batch_norm = nn.BatchNorm2d(out_chanels)
        
    def forward(self, x):
        return F.relu(self.batch_norm(self.conv(x)))

Inception block: contains various branches, output from these branches are concatenated to produce a final output in the ```forward()``` function

In [ ]:
class InceptionBlock(nn.Module):
    def __init__(
        self, 
        in_channels, 
        out_1x1,
        red_3x3,
        out_3x3,
        red_5x5,
        out_5x5,
        out_pool,
    ):
        super(InceptionBlock, self).__init__()
        self.branch1 = ConvBlock(in_channels, out_1x1, kernel_size=1)
        self.branch2 = nn.Sequential(
            ConvBlock(in_channels, red_3x3, kernel_size=1, padding=0),
            ConvBlock(red_3x3, out_3x3, kernel_size=3, padding=1),
        )
        self.branch3 = nn.Sequential(
            ConvBlock(in_channels, red_5x5, kernel_size=1),
            ConvBlock(red_5x5, out_5x5, kernel_size=5, padding=2),
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, padding=1, stride=1),
            ConvBlock(in_channels, out_pool, kernel_size=1),
        )
    
    def forward(self, x):
        branches = (self.branch1, self.branch2, self.branch3, self.branch4)
        return torch.cat([branch(x) for branch in branches], dim=1)

Auxiliary classifiers -- check setup in inceptionv3

In [ ]:
class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        # ? adaptive pooling necessary or use normal?
        self.pool = nn.AvgPool2d(kernel_size=5, stride=3)
        self.conv = ConvBlock(in_channels, 128, kernel_size=1)
        # in_features=2048 because we should flatten the input tensor which has shape of (batch, 4,4,128) so after flaten the tensor will be (batch, 4*4*128)
        # out_features=1024 this number from paper ??
        self.fc1 = nn.Linear(2048, 1024)
        # ? must this be included?
        # self.batch_norm = nn.BatchNorm1d(num_features=1024)
        self.dropout = nn.Dropout(p=0.7)
        self.fc2 = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.pool(x)
        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x) # dropout?
        x = self.fc2(x)
        return x

Full model

In [ ]:
class InceptionV1(nn.Module):
    def __init__(self, aux_logits=True, num_classes=1000):
        super(InceptionV1, self).__init__()
        self.aux_logits = aux_logits
        self.conv1 = ConvBlock(
            in_channels=3, 
            out_chanels=64,
            kernel_size=(7, 7),
            stride=(2, 2),
            padding=(3, 3),
        )
        self.conv2 = ConvBlock(64, 192, kernel_size=3, stride=1, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.inception3a = InceptionBlock(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = InceptionBlock(256, 128, 128, 192, 32, 96, 64)
        self.inception4a = InceptionBlock(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = InceptionBlock(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = InceptionBlock(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = InceptionBlock(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = InceptionBlock(528, 256, 160, 320, 32, 128, 128)
        self.inception5a = InceptionBlock(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = InceptionBlock(832, 384, 192, 384, 48, 128, 128)
        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, num_classes)
        
        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)
        else:
            self.aux1 = self.aux2 = None
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool(x)
        x = self.inception4a(x)
        
        if self.aux_logits and self.training:
            aux1 = self.aux1(x)
        
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        
        if self.aux_logits and self.training:
            aux2 = self.aux2(x)
        
        x = self.inception4e(x)
        x = self.maxpool(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc(x)
        
        if self.aux_logits and self.training:
            return aux1, aux2, x
        return x

Check model

In [ ]:
model = InceptionV1()
print(model)

Model_Summary(model, test_input.shape)

In [ ]:
# Test model: sanity check with dummy input
test_input = torch.randn(2, 3, 224, 224)
aux1, aux2, output = model(test_input)
print(output.shape)

### Helper functions

### Parameters

In [2]:
# Number of classes in the dataset
num_classes = 2
# Batch size for training (change depending on how much memory you have)
batch_size = 32
# Number of epochs to train for
# * paper: 100
num_epochs = 20

### Initialise model

In [ ]:
# move function to `initalise_models.py`
def inception(num_classes):

    # Hyperparameters
    WEIGHT_DECAY = 0.9                  # Decay term for RMSProp.
    MOMENTUM = 0.9                      # Momentum in RMSProp.
    EPSILON = 1.0                       # Epsilon term for RMSProp.
    INITIAL_LEARNING_RATE = 0.1         # Initial learning rate.
    NUM_EPOCHS_PER_DECAY = 30.0         # Epochs after which learning rate decays.
    LEARNING_RATE_DECAY_FACTOR = 0.16   # Learning rate decay factor.

    model = InceptionV3()
    # * set parameters correct?
    optimiser = optim.RMSprop(lr=INITIAL_LEARNING_RATE, momentum=MOMENTUM, eps=EPSILON, weight_decay=WEIGHT_DECAY)
    criterion = nn.CrossEntropyLoss()
    # * complete
    parameters = {"learning_rate": INITIAL_LEARNING_RATE, "momentum": MOMENTUM}

    return model, optimiser, criterion, parameters

### Load data

### Model

In [ ]:
# Hyperparameters
RMSPROP_DECAY = 0.9                # Decay term for RMSProp.
MOMENTUM = 0.9                     # Momentum in RMSProp.
RMSPROP_EPSILON = 1.0              # Epsilon term for RMSProp.
INITIAL_LEARNING_RATE = 0.1        # Initial learning rate.
NUM_EPOCHS_PER_DECAY = 30.0        # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.16  # Learning rate decay factor.